In [26]:
import pandas as pd
import matplotlib
# import geopandas as gpd
import datetime,os,re,sys,subprocess
from socket import gethostname

from simpledbf import Dbf5 as dbf
import datetime,os,re,sys,subprocess

In [27]:
#Traffic constants
SCRIPT_PATH             = sys.path[0] + "\\"
HWY_CREATEDAILY_CMD     = SCRIPT_PATH + "create-daily.s"
HWY_VOLFILES            = { 'AM':'am_vols.dbf', 'MD':'md_vols.dbf', 'PM':'pm_vols.dbf', 
                            'EV':'ev_vols.dbf', 'EA':'ea_vols.dbf', 'Daily':'daily_vols.dbf' }
HWY_ALLCOUNTYLINES      = 'All SF County Lines'
HWY_ROWS                = [ HWY_ALLCOUNTYLINES, 'SM County Line', 'Bridges', ['Bay Bridge',1], ['GG Bridge',1], 
                            ['San Rafael Bridge (WB,EB)',1], ['San Mateo Bridge (WB,EB)',1], ['Dumbarton Bridge (WB,EB)',1],
                            [ 'Carquinez Bridge',1], # ['Benicia-Martinez Bridge',1],
                            'Octavia @ Market (NB,SB)', '19th Ave @ Lincoln (NB,SB)',
                            'Geary/O\'Farrell @ VN (EB,WB)', 'Van Ness @ Geary (NB,SB)' ]
HWY_SCREENS             = { \
    "Bay Bridge":[
                    [ 52832, 52494 ],   # inbound
                    [ 52495, 52833 ],   # outbound
    ],
    "GG Bridge":[
                    [ 52267, 52425 ],   # inbound
                    [ 52426, 52268 ],   # outbound
    ],
    "San Rafael Bridge (WB,EB)":[
                    [ 103342, 8894 ],     # inbound = westbound
                    [ 8853, 103341 ],     # outbound = eastbound
    ],
    "San Mateo Bridge (WB,EB)":[
                    [ 7381, 7393 ],     # inbound = westbound
                    [ 7382, 7383 ],     # outbound = eastbound
    ],
    "Dumbarton Bridge (WB,EB)":[
                    [ 6938, 6921 ],     # inbound = westbound
                    [ 6922, 6939 ],     # outbound = eastbound
    ],
    "Carquinez Bridge":[
                    [ 12643, 12667, 
                      12643, 12634 ],   # inbound
                    [ 12637, 12642,
                      10482, 10483 ],   # outbound
    ],
    "Benicia-Martinez Bridge":[
                    [ 3131, 3136,
                      3131, 10247 ],    # inbound
                    [ 3135, 3130,
                      3132, 3142 ],     # outbound
    ],
    "SM County Line":[ # done by looking @ network and loading bayarea_county.shhp
                       # just documenting all cross points
                       # the big 3 (101, 280, 35) all match the roadway validation spreadsheet
                    [ 7678,  51171,     # inbound - 35 
                      7677,  33539,     # lake merced
                      51450, 33467,     # junipero serra
                      51130, 22527,     # saint charles
                      52270, 22515,     # de long
                      52234, 52271,     # 280 north
                      51137, 22513,     # santa cruz
                      52456, 51138,     # santa barbara
                      33704, 33454,     # shakespeare st
                      33704, 22482,     # rhine st
                      51142, 22482,     # flournoy st
                      51113, 22479,     # wilson st
                      51113, 22464,     # san jose ave
                      52461, 22464,     # goethe st
                      52462, 22465,     # rice st
                      52463, 22452,     # liebig st
                      52787, 21584,     # mission st
                      52792, 33702,     # irvington st
                      52469, 33702,     # acton st
                      52459, 21531,     # brunswick st
                      52790, 21531,     # oliver st
                      51019, 21500,     # whittier st
                      51019, 21493,     # hanover st
                      52774, 21493,     # lowell st
                      33603, 21461,     # guttenberg st
                      33844, 33351,     # pope st
                      51000, 33351,     # polaris way
                      52487, 51009,     # cordova st
                      52487, 21371,     # canyon way
                      51002, 21371,     # south hill blvd
                      52489, 21359,     # carter st
                      52482, 20421,     # geneva ave
                      52482, 20413,     # pasadena st
                      52483, 20412,     # castillo st
                      52481, 20411,     # pueblo str
                      52484, 20402,     # calgary st
                      52485, 20403,     # rio verde st
                      33605, 20373,     # velasco ave
                      33605, 20375,     # schwerin st
                      50995, 20306,     # bayshore
                      6986,  50856,     # tunnel ave
                      6985,  52492,     # alana way
                      40029, 52118,     # hwy 101nb
                      40003, 50852 ],   # harney way
                    [ 51171, 7678,      # outbound - 35
                      33549, 7677,      # lake merced
                      22528, 51449,     # alemany blvd
                      52480, 51449,     # junipero serra
                      22527, 51130,     # saint charles
                      52137, 52236,     # 280 s offramp
                      52137, 52235,     # another 280 s offramp
                      22515, 52270,     # de long
                      22513, 51137,     # santa cruz
                      51138, 52456,     # santa barbara
                      33454, 33704,     # shakespeare st
                      22482, 33704,     # rhine st
                      22482, 51142,     # flournoy st
                      22479, 51113,     # wilson st
                      22464, 51113,     # san jose ave
                      22464, 52461,     # goethe st
                      22465, 52462,     # rice st
                      22452, 52463,     # liebig st
                      21584, 52787,     # mission st
                      33702, 52792,     # irvington st
                      33702, 52469,     # acton st
                      21531, 52459,     # brunswick st
                      21531, 52790,     # oliver st
                      21500, 51019,     # whittier st
                      21493, 51019,     # hanover st
                      21493, 52774,     # lowell st
                      21461, 33603,     # guttenberg st
                      33351, 33844,     # pope st
                      33351, 51000,     # polaris way
                      51009, 52487,     # cordova st
                      21371, 52487,     # canyon way
                      21371, 51002,     # south hill blvd
                      21359, 52489,     # carter st
                      20421, 52482,     # geneva ave
                      20413, 52482,     # pasadena st
                      20412, 52483,     # castillo st
                      20411, 52481,     # pueblo str
                      20402, 52484,     # calgary st
                      20403, 52485,     # rio verde st
                      20373, 33605,     # velasco ave
                      20375, 33605,     # schwerin st
                      20306, 50995,     # bayshore
                      50856, 6986,      # tunnel ave
                      52492, 6985,      # alana way
                      52264, 7732,      # hwy 101sb
                      69013, 10689,		# hwy 101sb HOV
                      50852, 40003 ]    # harney way
    ],
    "Octavia @ Market (NB,SB)":[
                    [ 30756, 25858,     # inbound
                      30756, 53029 ],
                    [ 53029, 30756,     # oubound
                      53028, 30756 ]
    ],
    "Geary/O'Farrell @ VN (EB,WB)":[
                    [ 25196, 25192 ],   # inbound
                    [ 25197, 25213 ],   # outbound
    ],
    "Van Ness @ Geary (NB,SB)":[
                    [ 25195, 25197 ],   # nb
                    [ 25197, 30709 ],   # sb
    ],
    "19th Ave @ Lincoln (NB,SB)":[
                    [ 27372, 27374 ],   # nb
                    [ 27374, 27372 ],   # sb
    ],
}   

In [28]:
def runtpp(scriptname, outputfile, env=None):
    """ Runs the given tpp script if the given outputfile doesn't exist
    """
    try:
        if (runAll):
            raise Exception('runAll', 'runAll')
        else:
            os.stat(outputfile)
    except:
        # it doesn't exist, run it
        hostname = gethostname()
        fullenv = None
        if env:
            fullenv = os.environ
            fullenv.update(env)
        # dispatch it, no cube license
        if (hostname == 'Mason' or hostname == 'D90V07K1-okforgetit'):
            f = open('topsheet.tmp', 'w')
            f.write("runtpp " + scriptname + "\n")
            f.close()
            scriptname = scriptname.replace('/', '\\')
            dproc    = subprocess.Popen( "Y:/champ/util/bin/dispatch.bat topsheet.tmp ocean", env=fullenv) 
            dret     = dproc.wait()
        else:
            dproc    = subprocess.Popen( "runtpp " + scriptname, env=fullenv)    
            dret     = dproc.wait()
        print("dret = ",dret)
    return

In [29]:
def getTrafficScreens(timePeriod='Daily'):
    """
    """
    hwyfile     = HWY_VOLFILES[timePeriod]
    print(hwyfile)
    runtpp(HWY_CREATEDAILY_CMD, hwyfile)    # create it, if needed
    file_dbf = dbf('data/' + hwyfile)#to be changed later!!!!!!!!!!!
    hwydbf      = file_dbf.to_dataframe()
    abToScreen  = {}
    traffic     = {}
    if (timePeriod == 'Daily'):
        vol     = 'DAILY_TOT'
    else:
        vol     = 'TOTVOL'
    # build a reverse mapping for what we want
    ## We can use numpy or direct for key,value in dict.itmes() instead of so many loops
    for key,value in HWY_SCREENS.items():
        #inbound
        for index in range(0,len(value[0]),2):
            abToScreen[str(value[0][index])+" "+str(value[0][index+1])] = [ key, 0 ]
        #outbound
        for index in range(0,len(value[1]),2):
            abToScreen[str(value[1][index])+" "+str(value[1][index+1])] = [ key, 1 ]
            
    # print(abToScreen)
    # # # print(hwydbf)
    # # return 0
    for i in range(len(hwydbf)):
        rec = hwydbf.loc[i]
        if (rec['AB'] in abToScreen):
            screen,inout = abToScreen[rec['AB']][0],abToScreen[rec['AB']][1]
            if screen not in traffic: traffic[screen] = [ 0, 0 ] #1st inbound, 2nd outbound
            # if (screen == 'SM County Line'):
            #   print screen, inout, rec['AB'], int(rec[vol])
            traffic[screen][inout] = traffic[screen][inout] + rec[vol]
    # make them ints
    for screen in traffic:
        traffic[screen][0] = int(traffic[screen][0])
        traffic[screen][1] = int(traffic[screen][1])
        
    # aggregate
    traffic[HWY_ALLCOUNTYLINES] = [ 0, 0]
    traffic[HWY_ALLCOUNTYLINES][0] = \
         traffic['SM County Line'][0] + traffic['Bay Bridge'][0] + traffic['GG Bridge'][0]
    traffic[HWY_ALLCOUNTYLINES][1] = \
         traffic['SM County Line'][1] + traffic['Bay Bridge'][1] + traffic['GG Bridge'][1]
    return traffic

In [30]:
tr = getTrafficScreens()
tr
#HWY_VOLFILES            = { 'AM':'am_vols.dbf', 'MD':'md_vols.dbf', 'PM':'pm_vols.dbf', 
                            # 'EV':'ev_vols.dbf', 'EA':'ea_vols.dbf', 'Daily':'daily_vols.dbf' }

daily_vols.dbf
dret =  11


{'Dumbarton Bridge (WB,EB)': [47593, 67173],
 'SM County Line': [363893, 347446],
 'San Mateo Bridge (WB,EB)': [56901, 55669],
 'San Rafael Bridge (WB,EB)': [46671, 51624],
 'Carquinez Bridge': [67183, 64616],
 'Van Ness @ Geary (NB,SB)': [8321, 9927],
 "Geary/O'Farrell @ VN (EB,WB)": [24836, 17084],
 '19th Ave @ Lincoln (NB,SB)': [31947, 27505],
 'Octavia @ Market (NB,SB)': [39058, 37177],
 'GG Bridge': [68204, 70430],
 'Bay Bridge': [137180, 152869],
 'All SF County Lines': [569277, 570745]}

In [6]:
df = pd.DataFrame(data=tr).T.reset_index()
df.iloc[:,1:3]

,0,1
0,47593,67173
1,363893,347446
2,56901,55669
3,46671,51624
4,67183,64616
5,8321,9927
6,24836,17084
7,31947,27505
8,39058,37177
9,68204,70430


In [8]:
time = ['AM','MD','PM','EV','EA']
dow_df=pd.DataFrame()
for t in time:
    dict_t = getTrafficScreens(t)
    df_t = pd.DataFrame(data=dict_t)
    tra_pm = df_t.transpose().reset_index()
    index_order = [11,1,10,9,3,2,0,4,8,7,6,5]
    tra_pm = tra_pm.reindex(index_order,  axis=0)
    tra_pm.columns = ['Lines',f'{t} In',f'{t} Out']
    # print(tra_pm)
    if t=='AM':
        dow_df = tra_pm.copy()
    else:
        dow_df = pd.merge(dow_df,tra_pm)
    
dow_df

am_vols.dbf
dret =  11
md_vols.dbf
dret =  11
pm_vols.dbf
dret =  11
ev_vols.dbf
dret =  11
ea_vols.dbf
dret =  11


,Lines,AM In,AM Out,MD In,MD Out,PM In,PM Out,EV In,EV Out,EA In,EA Out
0,All SF County Lines,113438,88258,206033,197754,105235,119126,116901,146684,27666,18916
1,SM County Line,62914,60141,132599,124727,74870,69099,82879,79594,10630,13883
2,Bay Bridge,31223,22462,43688,47719,23071,32844,26529,45611,12668,4230
3,GG Bridge,19301,5655,29746,25308,7294,17183,7493,21479,4368,803
4,"San Rafael Bridge (WB,EB)",9377,11120,14923,17920,10285,10546,10351,8130,1733,3905
5,"San Mateo Bridge (WB,EB)",16432,7409,17464,16333,9206,15863,9667,14837,4130,1225
6,"Dumbarton Bridge (WB,EB)",17819,6988,11761,20189,7431,19146,6654,19537,3927,1311
7,Carquinez Bridge,15650,8426,23337,21653,10114,14827,11436,17646,6643,2062
8,"Octavia @ Market (NB,SB)",5413,6375,14084,12774,7083,5753,10027,9173,2448,3100
9,"19th Ave @ Lincoln (NB,SB)",5320,5452,11713,10367,6360,5355,7649,5225,903,1103


In [12]:
dow_df.index = dow_df['Lines']
dow_df.drop('Lines',axis =1, inplace=True)
dow_df

,AM In,AM Out,MD In,MD Out,PM In,PM Out,EV In,EV Out,EA In,EA Out
Lines,,,,,,,,,,
All SF County Lines,113438,88258,206033,197754,105235,119126,116901,146684,27666,18916
SM County Line,62914,60141,132599,124727,74870,69099,82879,79594,10630,13883
Bay Bridge,31223,22462,43688,47719,23071,32844,26529,45611,12668,4230
GG Bridge,19301,5655,29746,25308,7294,17183,7493,21479,4368,803
"San Rafael Bridge (WB,EB)",9377,11120,14923,17920,10285,10546,10351,8130,1733,3905
"San Mateo Bridge (WB,EB)",16432,7409,17464,16333,9206,15863,9667,14837,4130,1225
"Dumbarton Bridge (WB,EB)",17819,6988,11761,20189,7431,19146,6654,19537,3927,1311
Carquinez Bridge,15650,8426,23337,21653,10114,14827,11436,17646,6643,2062
"Octavia @ Market (NB,SB)",5413,6375,14084,12774,7083,5753,10027,9173,2448,3100


In [17]:
even_columns = dow_df.iloc[:, ::2]
even_sum = even_columns.sum(axis=1)

# Get the odd columns and calculate their sum
odd_columns = dow_df.iloc[:, 1::2]
odd_sum = odd_columns.sum(axis=1)

# Add the sum of even columns as a new column 'Even_Column_Sum'
dow_df['Total Indbound'] = even_sum

# Add the sum of odd columns as a new column 'Odd_Column_Sum'
dow_df['Total Outbound'] = odd_sum

# Now df contains the sums of alternate columns in new columns
dow_df.reset_index(inplace=True)
dow_df

,Lines,AM In,AM Out,MD In,MD Out,PM In,PM Out,EV In,EV Out,EA In,EA Out,Total Indbound,Total Outbound
0,All SF County Lines,113438,88258,206033,197754,105235,119126,116901,146684,27666,18916,2846365,2853690
1,SM County Line,62914,60141,132599,124727,74870,69099,82879,79594,10630,13883,1819460,1737220
2,Bay Bridge,31223,22462,43688,47719,23071,32844,26529,45611,12668,4230,685895,764330
3,GG Bridge,19301,5655,29746,25308,7294,17183,7493,21479,4368,803,341010,352140
4,"San Rafael Bridge (WB,EB)",9377,11120,14923,17920,10285,10546,10351,8130,1733,3905,233345,258105
5,"San Mateo Bridge (WB,EB)",16432,7409,17464,16333,9206,15863,9667,14837,4130,1225,284495,278335
6,"Dumbarton Bridge (WB,EB)",17819,6988,11761,20189,7431,19146,6654,19537,3927,1311,237960,335855
7,Carquinez Bridge,15650,8426,23337,21653,10114,14827,11436,17646,6643,2062,335900,323070
8,"Octavia @ Market (NB,SB)",5413,6375,14084,12774,7083,5753,10027,9173,2448,3100,195275,185875
9,"19th Ave @ Lincoln (NB,SB)",5320,5452,11713,10367,6360,5355,7649,5225,903,1103,159725,137510


In [23]:
cols = dow_df.columns 
for col in cols[1:]:
    dow_df[col] = dow_df[col].apply(lambda x: '{:,.0f}'.format(x)) 
markdown_table = dow_df.to_markdown(index=False).split('\n')
header_row = markdown_table[0]
header_row = '| ' + ' | '.join(f'**{header.strip()}**' for header in header_row.split('|')[1:-1]) + ' |'
markdown_table[0] = header_row
markdown_table[1] = markdown_table[1].replace('|', ':|:').replace('::',':').replace(':|:',':|')[1:-1]
markdown_table = '\n'.join(markdown_table)
print(markdown_table)
with open('traffic_dow.md', 'w') as f:
        f.write(markdown_table)

| **Lines** | **AM In** | **AM Out** | **MD In** | **MD Out** | **PM In** | **PM Out** | **EV In** | **EV Out** | **EA In** | **EA Out** | **Total Indbound** | **Total Outbound** |
|-----------------------------:|--------:|---------:|--------:|---------:|--------:|---------:|--------:|---------:|--------:|---------:|-----------------:|-----------------:
| All SF County Lines          | 113,438 | 88,258   | 206,033 | 197,754  | 105,235 | 119,126  | 116,901 | 146,684  | 27,666  | 18,916   | 2,846,365        | 2,853,690        |
| SM County Line               | 62,914  | 60,141   | 132,599 | 124,727  | 74,870  | 69,099   | 82,879  | 79,594   | 10,630  | 13,883   | 1,819,460        | 1,737,220        |
| Bay Bridge                   | 31,223  | 22,462   | 43,688  | 47,719   | 23,071  | 32,844   | 26,529  | 45,611   | 12,668  | 4,230    | 685,895          | 764,330          |
| GG Bridge                    | 19,301  | 5,655    | 29,746  | 25,308   | 7,294   | 17,183   | 7,493   | 21,479   |

##### This can be used to check the markdown!!

In [8]:
def trafficMaps(timePeriod = 'Daily'):
    hwyfile     = HWY_VOLFILES[timePeriod]
    file_dbf = dbf('data/' + hwyfile)#to be changed later!!!!!!!!!!!
    hwydbf      = file_dbf.to_dataframe()
    if timePeriod!='Daily':
        data = hwydbf[['AB','DA','SR2','SR3','COM','TRK','BUS','TNC','AUTOVOL','TOTVOL']]
    else:
        data = hwydbf[['AB','DAILY_DA', 'DAILY_SR2', 'DAILY_SR3','DAILY_COM', 'DAILY_TRK', 
       'DAILY_BUS', 'DAILY_TNC', 'DAILY_AUTO', 'DAILY_TOT']]

    # nonOdist = data[data['DISTANCE']!=0]
    data.to_csv('traffic_map_'+timePeriod+'.csv',index=False,header=['AB','DA','SR2','SR3','COM','TRK','BUS','TNC','AUTOVOL','TOTVOL'])

In [32]:
hwyfile     = HWY_VOLFILES['Daily']
file_dbf = dbf('data/' + hwyfile)#to be changed later!!!!!!!!!!!
hwydbf      = file_dbf.to_dataframe()
hwydbf.iloc[0:5]

,AB,A,B,AT,FT,LANE,CAP,SPEED,DISTANCE,DAILY_DA,...,DAILY_SR3,DAILY_TRK,DAILY_COM,DAILY_BUS,DAILY_TNC,DAILY_AUTO,DAILY_PAX,DAILY_TOT,DAILY_PCE,MTYPE
0,1 20368,1,20368,3,6,7,2000,0,0.01410,359.41882,...,84.22974,2.01512,126.49341,0.0,10.09473,596.82880,960.58337,735.43207,799.68634,SF
1,1 20373,1,20373,3,6,7,2000,0,0.07441,0.00000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,SF
2,1 20375,1,20375,3,6,7,2000,0,0.10080,0.00000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,SF
3,1 20408,1,20408,3,6,7,2000,0,0.03971,0.00000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,SF
4,1 33349,1,33349,3,6,7,2000,0,0.04826,89.24449,...,26.25445,0.22901,32.39344,0.0,2.69545,165.52925,281.19568,200.84715,217.15839,SF


In [34]:
hwydbf[['AB','DAILY_DA','DAILY_SR2', 'DAILY_SR3', 'DAILY_TRK', 'DAILY_COM','DAILY_BUS', 'DAILY_TNC', 'DAILY_AUTO']]

,AB,DAILY_DA,DAILY_SR2,DAILY_SR3,DAILY_TRK,DAILY_COM,DAILY_BUS,DAILY_TNC,DAILY_AUTO
0,1 20368,359.41882,153.18025,84.22974,2.01512,126.49341,0.00,10.09473,596.82880
1,1 20373,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00000,0.00000
2,1 20375,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00000,0.00000
3,1 20408,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00000,0.00000
4,1 33349,89.24449,50.03031,26.25445,0.22901,32.39344,0.00,2.69545,165.52925
...,...,...,...,...,...,...,...,...,...
62387,103998 102738,8173.05957,2108.64771,960.45868,207.10678,1809.25610,12.50,68.45824,11242.16602
62388,103998 102741,5908.80518,1677.19629,685.25549,125.63978,2438.29346,0.00,51.94152,8271.25684
62389,103999 4026,4249.88770,1163.91309,434.04291,76.73005,1082.05701,124.93,41.81718,5847.84326
62390,103999 4030,3547.83105,878.74908,370.53577,84.60525,1003.64844,0.00,29.98256,4797.11572


In [70]:
hwydbf.columns

Index(['AB', 'A', 'B', 'AT', 'FT', 'LANE', 'CAP', 'SPEED', 'DISTANCE',
       'DAILY_DA', 'DAILY_SR2', 'DAILY_SR3', 'DAILY_TRK', 'DAILY_COM',
       'DAILY_BUS', 'DAILY_TNC', 'DAILY_AUTO', 'DAILY_PAX', 'DAILY_TOT',
       'DAILY_PCE', 'MTYPE'],
      dtype='object')

In [31]:
def getTypeTrafficScreens(timePeriod='Daily'):
    """
    """
    hwyfile     = HWY_VOLFILES[timePeriod]
    print(hwyfile)
    runtpp(HWY_CREATEDAILY_CMD, hwyfile)    # create it, if needed
    file_dbf = dbf('data/' + hwyfile)#to be changed later!!!!!!!!!!!
    hwydbf      = file_dbf.to_dataframe()
    abToScreen  = {}
    v_class=[]
    volume = ['DAILY_DA','DAILY_SR2', 'DAILY_SR3', 'DAILY_TRK', 'DAILY_COM','DAILY_BUS', 'DAILY_TNC', 'DAILY_AUTO']
    # build a reverse mapping for what we want
    ## We can use numpy or direct for key,value in dict.itmes() instead of so many loops
    for key,value in HWY_SCREENS.items():
        #inbound
        for index in range(0,len(value[0]),2):
            abToScreen[str(value[0][index])+" "+str(value[0][index+1])] = [ key, 0 ]
        #outbound
        for index in range(0,len(value[1]),2):
            abToScreen[str(value[1][index])+" "+str(value[1][index+1])] = [ key, 1 ]
    for vol in volume:
        traffic     = {}       
        for i in range(len(hwydbf)):
            rec = hwydbf.loc[i]
            if (rec['AB'] in abToScreen):
                screen,inout = abToScreen[rec['AB']][0],abToScreen[rec['AB']][1]
                if screen not in traffic: traffic[screen] = [ 0, 0 ] #1st inbound, 2nd outbound
                # if (screen == 'SM County Line'):
                #   print screen, inout, rec['AB'], int(rec[vol])
                traffic[screen][inout] = traffic[screen][inout] + rec[vol]
        # make them ints
        for screen in traffic:
            traffic[screen][0] = int(traffic[screen][0])
            traffic[screen][1] = int(traffic[screen][1])

        # aggregate
        traffic[HWY_ALLCOUNTYLINES] = [ 0, 0]
        traffic[HWY_ALLCOUNTYLINES][0] = \
            traffic['SM County Line'][0] + traffic['Bay Bridge'][0] + traffic['GG Bridge'][0]
        traffic[HWY_ALLCOUNTYLINES][1] = \
            traffic['SM County Line'][1] + traffic['Bay Bridge'][1] + traffic['GG Bridge'][1]
        v_class.append(traffic)
    return v_class

In [7]:
dicts = getTypeTrafficScreens()
df1 = [pd.DataFrame(d) for d in dicts]
pd.concat(df1,ignore_index=True).T
#['DAILY_DA','DAILY_SR2', 'DAILY_SR3', 'DAILY_TRK', 'DAILY_COM','DAILY_BUS', 'DAILY_TNC', 'DAILY_AUTO']

daily_vols.dbf
dret =  11


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
"Dumbarton Bridge (WB,EB)",31798,48055,7508,8412,2904,3347,628,1266,4601,5834,52,105,98,150,42212,59815
SM County Line,193909,185066,41292,38223,16701,15880,6823,6305,99968,97067,1212,1095,3984,3808,251903,239169
"San Mateo Bridge (WB,EB)",42313,40150,8001,6326,2663,2462,1402,2923,2374,3701,19,17,125,87,52979,48939
"San Rafael Bridge (WB,EB)",31880,35462,5508,5613,2085,2235,786,1155,6322,7064,34,35,52,57,39474,43311
Carquinez Bridge,47778,44698,6096,5953,3159,3260,3999,3850,6004,6651,102,146,42,56,57034,53912
"Van Ness @ Geary (NB,SB)",3438,4069,687,971,244,334,189,193,2366,2648,314,321,1079,1388,4370,5375
"Geary/O'Farrell @ VN (EB,WB)",12279,7918,2071,1305,711,429,458,417,6027,4640,389,389,2899,1984,15062,9653
"19th Ave @ Lincoln (NB,SB)",20045,16337,4075,3793,1549,1470,575,460,5051,4849,213,208,436,386,25670,21600
"Octavia @ Market (NB,SB)",24460,24894,4350,4126,1761,1588,811,709,6740,5298,0,0,934,559,30572,30608
GG Bridge,37559,44987,6892,6820,2328,2344,2340,2097,18580,13640,211,198,292,341,46780,54153


In [58]:
df1 = [pd.DataFrame(d) for d in dicts]
df2 = pd.concat(df1,ignore_index=True).T
mtables = df2.iloc[[1,9,10,11]]
mtables

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
SM County Line,193909,185066,41292,38223,16701,15880,6823,6305,99968,97067,1212,1095,3984,3808,251903,239169
GG Bridge,37559,44987,6892,6820,2328,2344,2340,2097,18580,13640,211,198,292,341,46780,54153
Bay Bridge,100345,108251,16611,18878,7698,7432,2005,5156,7408,9567,452,636,2659,2945,124655,134563
All SF County Lines,331813,338304,64795,63921,26727,25656,11168,13558,125956,120274,1875,1929,6935,7094,423338,427885


In [69]:
mtables.index

Index(['SM County Line', 'GG Bridge', 'Bay Bridge', 'All SF County Lines'], dtype='object')

In [56]:
inbound = mtables.iloc[0,::2].T.reset_index(drop=True)
outbound = mtables.iloc[0,1::2].T.reset_index(drop=True)
SM_df = pd.concat([inbound,outbound],axis=1)
SM_df.index = ['DA','SR2', 'SR3', 'TRK', 'COM','BUS', 'TNC', 'AUTO']
SM_df.reset_index(drop=False,inplace=True)
SM_df.columns = ['Vehicle_Class','Inbound','Outbound']
SM_df

,Vehicle_Class,Inbound,Outbound
0,DA,193909,185066
1,SR2,41292,38223
2,SR3,16701,15880
3,TRK,6823,6305
4,COM,99968,97067
5,BUS,1212,1095
6,TNC,3984,3808
7,AUTO,251903,239169


In [57]:
markdown_table = SM_df.to_markdown(index=False).split('\n')
header_row = markdown_table[0]
header_row = '| ' + ' | '.join(f'**{header.strip()}**' for header in header_row.split('|')[1:-1]) + ' |'
markdown_table[0] = header_row
markdown_table[1] = markdown_table[1].replace('|', ':|:').replace('::',':')[1:-1]
markdown_table = '\n'.join(markdown_table)
print(markdown_table)

| **Vehicle_Class** | **Inbound** | **Outbound** |
|:----------------:|:----------:|:-----------:|
| DA              |    193909 |     185066 |
| SR2             |     41292 |      38223 |
| SR3             |     16701 |      15880 |
| TRK             |      6823 |       6305 |
| COM             |     99968 |      97067 |
| BUS             |      1212 |       1095 |
| TNC             |      3984 |       3808 |
| AUTO            |    251903 |     239169 |


| **Vehicle_Class** | **Inbound** | **Outbound** |
|:----------------:|:----------:|:-----------:|
| DA              |    193909 |     185066 |
| SR2             |     41292 |      38223 |
| SR3             |     16701 |      15880 |
| TRK             |      6823 |       6305 |
| COM             |     99968 |      97067 |
| BUS             |      1212 |       1095 |
| TNC             |      3984 |       3808 |
| AUTO            |    251903 |     239169 |

In [54]:
def lineFiles(dicts,line='All SF County Lines'):
    df1 = [pd.DataFrame(d) for d in dicts]
    df2 = pd.concat(df1,ignore_index=True).T
    mtables = df2.iloc[[1,9,10,11]]
    inbound = mtables.loc['SM County Line',::2].T.reset_index(drop=True)
    outbound = mtables.loc['SM County Line',1::2].T.reset_index(drop=True)
    df = pd.concat([inbound,outbound],axis=1)
    df.index = ['DA','SR2', 'SR3', 'TRK', 'COM','BUS', 'TNC', 'AUTO']
    df.columns = ['Inbound','Outbound']
    sum_row = df.iloc[0:7].sum()
    # Convert the sum to a DataFrame and transpose it to make it a row
    sum_row_df = pd.DataFrame([sum_row])

    df = pd.concat([df, sum_row_df],ignore_index=True)
    df.index = ['DA','SR2', 'SR3', 'TRK', 'COM','BUS', 'TNC', 'AUTO','TOTAL']
    df.reset_index(drop=False,inplace=True)
    df.columns = ['Class','Inbound','Outbound']
    markdown_table = df.to_markdown(index=False).split('\n')
    header_row = markdown_table[0]
    header_row = '| ' + ' | '.join(f'**{header.strip()}**' for header in header_row.split('|')[1:-1]) + ' |'
    markdown_table[0] = header_row
    markdown_table[1] = markdown_table[1].replace('|', ':|:').replace('::',':')[1:-1]
    markdown_table = '\n'.join(markdown_table)
    # print(markdown_table)
    with open('traffic_'+line+'.md', 'w') as f:
        f.write(markdown_table)

In [55]:
dicts = getTypeTrafficScreens()
majorLines = ['SM County Line', 'GG Bridge', 'Bay Bridge', 'All SF County Lines']
for line in majorLines:
    lineFiles(dicts,line)

daily_vols.dbf
dret =  11


In [53]:
df1 = [pd.DataFrame(d) for d in dicts]
df2 = pd.concat(df1,ignore_index=True).T
mtables = df2.iloc[[1,9,10,11]]
inbound = mtables.loc['SM County Line',::2].T.reset_index(drop=True)
outbound = mtables.loc['SM County Line',1::2].T.reset_index(drop=True)
df = pd.concat([inbound,outbound],axis=1)
df.index = ['DA','SR2', 'SR3', 'TRK', 'COM','BUS', 'TNC', 'AUTO']
df.columns = ['Inbound','Outbound']
sum_row = df.iloc[0:7].sum()
# Convert the sum to a DataFrame and transpose it to make it a row
sum_row_df = pd.DataFrame([sum_row])

df = pd.concat([df, sum_row_df],ignore_index=True)
df.index = ['DA','SR2', 'SR3', 'TRK', 'COM','BUS', 'TNC', 'AUTO','TOTAL']
df

,Inbound,Outbound
DA,193909,185066
SR2,41292,38223
SR3,16701,15880
TRK,6823,6305
COM,99968,97067
BUS,1212,1095
TNC,3984,3808
AUTO,251903,239169
TOTAL,363889,347444
